In [2]:
# CSL Paper: Dimensional speech emotion recognition from acoustic and text
# Changelog:
# 2019-09-01: initial version
# 2019-10-06: optimizer MTL parameters with linear search (in progress)
# 2019-12-21: update file for CSL journal
#             this python use following feature: mean, std, silence

import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

#from keras_self_attention import SeqSelfAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from attention_helper import AttentionDecoder
#from read_csv import load_features

import random as rn
import tensorflow as tf

# import gensim
# from gensim.models import Word2Vec
# from gensim.models.keyedvectors import KeyedVectors
import codecs

Using TensorFlow backend.


In [3]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# x_text = np.load('../improv_loso/data/x_text.npy')

In [4]:
# x_text.shape

In [4]:
# load data - label
list_path_train = '../improv_loso/data/data_mspin_train.csv'
list_train = pd.read_csv(list_path_train, index_col=None)

# vad_train = [list_train['v'], list_train['a'], list_train['d']]
# vad_train = np.array(vad_train).T

list_path_test = '../improv_loso/data/data_mspin_test.csv'
list_test = pd.read_csv(list_path_test, index_col=None)

## Text processing train
list_file = pd.concat([list_train, list_test])
text = list_file['transcript']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
nb_words = len(word_index) + 1
print('Found %s unique tokens' % len(word_index))

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_text = []
x_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

# x_text = np.load('improv_loso/data/x_text.npy')
# x_train_text = x_text[:5744] 
# x_test_text = x_text[5744:]

# vad = np.load('improv_loso/data/vad.npy')
# vad_train = vad[:5744]
# vad_test = vad[5744:]

EMBEDDING_DIM = 300
# nb_words = 3438
# MAX_SEQUENCE_LENGTH = 300

#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
file_loc =  '../../../dimensionalSM/data/crawl-300d-2M-subword.vec'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

Found 3427 unique tokens
../../../dimensionalSM/data/crawl-300d-2M-subword.vec
G Word embeddings: 2000000
G Null word embeddings: 275


In [5]:
MAX_SEQUENCE_LENGTH

300

In [6]:
vad = np.load('../improv_loso/data/vad.npy')

In [7]:
# standardization
scaled_feature = False

# set Dropout
do = 0.3

# standardized label
scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad


# Concordance correlation coefficient (CCC)-based loss function
# using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [8]:
# split train/test
split = 5733
earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)

In [9]:
# API model, if use RNN, first two rnn layer must return_sequences=True
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model1 = text_model1()

In [10]:
# main function for for LOSO
def main(time):
    model_1 = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [x_text[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [x_text[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict(x_text[4586:], batch_size=32)
    np.save('5733npy-fasttext/text-fasttext_5733-npy'+str(time)+'.npy',  
             np.array(predict).reshape(3, 2580).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('5733npy-fasttext/text-fasttext_5733.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 91s 20ms/step - loss: 2.5412 - v_loss: 0.7435 - a_loss: 0.8746 - d_loss: 0.9145 - v_ccc: 0.2542 - a_ccc: 0.1219 - d_ccc: 0.0826 - val_loss: 2.6863 - val_v_loss: 0.8694 - val_a_loss: 0.9133 - val_d_loss: 0.9010 - val_v_ccc: 0.1296 - val_a_ccc: 0.0862 - val_d_ccc: 0.0979
Epoch 2/50
4586/4586 [==============================] - 89s 19ms/step - loss: 1.9684 - v_loss: 0.5564 - a_loss: 0.6842 - d_loss: 0.7316 - v_ccc: 0.4435 - a_ccc: 0.3172 - d_ccc: 0.2708 - val_loss: 2.7177 - val_v_loss: 0.8812 - val_a_loss: 0.9054 - val_d_loss: 0.9289 - val_v_ccc: 0.1176 - val_a_ccc: 0.0944 - val_d_ccc: 0.0704
Epoch 3/50
4586/4586 [==============================] - 91s 20ms/step - loss: 1.7417 - v_loss: 0.4798 - a_loss: 0.6116 - d_loss: 0.6500 - v_ccc: 0.5195 - a_ccc: 0.3883 - d_ccc: 0.3506 - val_loss: 2.6370 - val_v_loss: 0

Epoch 7/50
4586/4586 [==============================] - 91s 20ms/step - loss: 1.2038 - v_loss: 0.3255 - a_loss: 0.4269 - d_loss: 0.4518 - v_ccc: 0.6743 - a_ccc: 0.5733 - d_ccc: 0.5485 - val_loss: 2.7788 - val_v_loss: 0.8634 - val_a_loss: 0.9230 - val_d_loss: 0.9905 - val_v_ccc: 0.1348 - val_a_ccc: 0.0772 - val_d_ccc: 0.0092
Epoch 8/50
4586/4586 [==============================] - 91s 20ms/step - loss: 1.1401 - v_loss: 0.3141 - a_loss: 0.4082 - d_loss: 0.4213 - v_ccc: 0.6868 - a_ccc: 0.5928 - d_ccc: 0.5803 - val_loss: 2.6636 - val_v_loss: 0.8411 - val_a_loss: 0.8995 - val_d_loss: 0.9205 - val_v_ccc: 0.1576 - val_a_ccc: 0.1000 - val_d_ccc: 0.0788
Epoch 9/50
4586/4586 [==============================] - 91s 20ms/step - loss: 1.0651 - v_loss: 0.2972 - a_loss: 0.3799 - d_loss: 0.3932 - v_ccc: 0.7045 - a_ccc: 0.6216 - d_ccc: 0.6088 - val_loss: 2.7377 - val_v_loss: 0.8560 - val_a_loss: 0.9048 - val_d_loss: 0.9750 - val_v_ccc: 0.1423 - val_a_ccc: 0.0954 - val_d_ccc: 0.0246
Epoch 10/50
4586/4586 

4586/4586 [==============================] - 90s 20ms/step - loss: 1.0202 - v_loss: 0.2810 - a_loss: 0.3629 - d_loss: 0.3750 - v_ccc: 0.7180 - a_ccc: 0.6376 - d_ccc: 0.6242 - val_loss: 2.7084 - val_v_loss: 0.8390 - val_a_loss: 0.9085 - val_d_loss: 0.9596 - val_v_ccc: 0.1594 - val_a_ccc: 0.0918 - val_d_ccc: 0.0404
Epoch 11/50
4586/4586 [==============================] - 90s 20ms/step - loss: 0.9512 - v_loss: 0.2638 - a_loss: 0.3371 - d_loss: 0.3553 - v_ccc: 0.7373 - a_ccc: 0.6646 - d_ccc: 0.6469 - val_loss: 2.7629 - val_v_loss: 0.8713 - val_a_loss: 0.9137 - val_d_loss: 0.9764 - val_v_ccc: 0.1269 - val_a_ccc: 0.0865 - val_d_ccc: 0.0237
Epoch 12/50
1433/1433 [==============================] - 8s 5ms/step
[0.24026024341583252, 0.20958322286605835, 0.1874631941318512]
5
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 94s 20ms/step - loss: 2.5734 - v_loss: 0.7437 - a_loss: 0.9027 - d_loss: 0.9248 - v_ccc: 0.2549 - a_ccc: 0.0956 - d_ccc:

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4586/4586 [==============================] - 89s 20ms/step - loss: 1.2118 - v_loss: 0.3312 - a_loss: 0.4324 - d_loss: 0.4500 - v_ccc: 0.6694 - a_ccc: 0.5682 - d_ccc: 0.5505 - val_loss: 2.7006 - val_v_loss: 0.8321 - val_a_loss: 0.9023 - val_d_loss: 0.9634 - val_v_ccc: 0.1659 - val_a_ccc: 0.0975 - val_d_ccc: 0.0359
Epoch 8/50
4586/4586 [==============================] - 89s 19ms/step - loss: 1.1521 - v_loss: 0.3180 - a_loss: 0.4072 - d_loss: 0.4257 - v_ccc: 0.6812 - a_ccc: 0.5919 - d_ccc: 0.5748 - val_loss: 2.7826 - val_v_loss: 0.8753 - val_a_loss: 0.9202 - val_d_loss: 0.9849 - val_v_ccc: 0.1231 - val_a_ccc: 0.0795 - val_d_ccc: 0.0149
Epoch 9/50
4586/4586 [==============================] - 89s 19ms/step - loss: 1.0817 - v_loss: 0.2967 - a_loss: 0.3858 - d_loss: 0.3976 - v_ccc: 0.7025 - a_ccc: 0.6138 - d_ccc: 0.6020 - val_loss: 2.7262 - val_v_loss: 0.8651 - val_a_loss: 0.9054 - val_d_loss: 0.9538 - val_v_ccc: 0.1333 - val_a_ccc: 0.0948 - val_d_ccc: 0.0457
Epoch 10/50
4586/4586 [==========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4586/4586 [==============================] - 92s 20ms/step - loss: 1.3206 - v_loss: 0.3622 - a_loss: 0.4634 - d_loss: 0.4952 - v_ccc: 0.6386 - a_ccc: 0.5362 - d_ccc: 0.5046 - val_loss: 2.7218 - val_v_loss: 0.8475 - val_a_loss: 0.9103 - val_d_loss: 0.9627 - val_v_ccc: 0.1511 - val_a_ccc: 0.0898 - val_d_ccc: 0.0373
Epoch 7/50
4586/4586 [==============================] - 93s 20ms/step - loss: 1.2137 - v_loss: 0.3362 - a_loss: 0.4262 - d_loss: 0.4506 - v_ccc: 0.6633 - a_ccc: 0.5733 - d_ccc: 0.5498 - val_loss: 2.7501 - val_v_loss: 0.8455 - val_a_loss: 0.9186 - val_d_loss: 0.9841 - val_v_ccc: 0.1527 - val_a_ccc: 0.0816 - val_d_ccc: 0.0155
Epoch 8/50
4586/4586 [==============================] - 92s 20ms/step - loss: 1.1494 - v_loss: 0.3132 - a_loss: 0.4146 - d_loss: 0.4259 - v_ccc: 0.6863 - a_ccc: 0.5867 - d_ccc: 0.5776 - val_loss: 2.7217 - val_v_loss: 0.8671 - val_a_loss: 0.8839 - val_d_loss: 0.9683 - val_v_ccc: 0.1314 - val_a_ccc: 0.1158 - val_d_ccc: 0.0311
Epoch 9/50
4586/4586 [===========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4586/4586 [==============================] - 92s 20ms/step - loss: 1.9876 - v_loss: 0.5623 - a_loss: 0.6909 - d_loss: 0.7313 - v_ccc: 0.4368 - a_ccc: 0.3085 - d_ccc: 0.2670 - val_loss: 2.5793 - val_v_loss: 0.8161 - val_a_loss: 0.8724 - val_d_loss: 0.8872 - val_v_ccc: 0.1820 - val_a_ccc: 0.1269 - val_d_ccc: 0.1118
Epoch 3/50
4586/4586 [==============================] - 91s 20ms/step - loss: 1.7593 - v_loss: 0.4833 - a_loss: 0.6132 - d_loss: 0.6667 - v_ccc: 0.5174 - a_ccc: 0.3878 - d_ccc: 0.3355 - val_loss: 2.7744 - val_v_loss: 0.8330 - val_a_loss: 0.9846 - val_d_loss: 0.9540 - val_v_ccc: 0.1653 - val_a_ccc: 0.0147 - val_d_ccc: 0.0455
Epoch 4/50
4586/4586 [==============================] - 84s 18ms/step - loss: 1.5686 - v_loss: 0.4348 - a_loss: 0.5485 - d_loss: 0.5813 - v_ccc: 0.5648 - a_ccc: 0.4493 - d_ccc: 0.4172 - val_loss: 2.6057 - val_v_loss: 0.8362 - val_a_loss: 0.8749 - val_d_loss: 0.8920 - val_v_ccc: 0.1625 - val_a_ccc: 0.1246 - val_d_ccc: 0.1072
Epoch 5/50
4586/4586 [===========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
# hist1 = model1.fit(x_text[:split], vad[:split].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate(x_text[split:], vad[split:].T.tolist())
# print(eval_metrik1[-3:])

# # make prediction
# predict = model1.predict(x_text[4596:], batch_size=32)
# np.save('text_glove_npy_loso', 
#          np.array(predict).reshape(3, 2570).T)